In [3]:
pip install mlflow

   ---------------------------------------- 0.0/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.2/26.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.7 MB 3.5 MB/s eta 0:00:08
    --------------------------------------- 0.4/26.7 MB 3.1 MB/s eta 0:00:09
    --------------------------------------- 0.5/26.7 MB 2.8 MB/s eta 0:00:10
    --------------------------------------- 0.7/26.7 MB 3.0 MB/s eta 0:00:09
   - -------------------------------------- 0.8/26.7 MB 2.8 MB/s eta 0:00:10
   - -------------------------------------- 1.0/26.7 MB 2.9 MB/s eta 0:00:09
   - -------------------------------------- 1.1/26.7 MB 3.0 MB/s eta 0:00:09
   - -------------------------------------- 1.3/26.7 MB 3.0 MB/s eta 0:00:09
   -- ------------------------------------- 1.4/26.7 MB 3.1 MB/s eta 0:00:09
   -- ------------------------------------- 1.6/26.7 MB 3.1 MB/s eta 0:00:09
   -- ------------------------------------- 1.8/26.7 MB 3.2 MB/s eta 0:00:08
   -- -------

In [16]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import mlflow3
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_validate

# IrisDataProcessor 
class IrisDataProcessor:
    def __init__(self):
        
        self.iris_data = load_iris()
        
        self.data = pd.DataFrame(data=self.iris_data.data, columns=self.iris_data.feature_names)
        self.data['target'] = self.iris_data.target  # Add target column
        
        # StandardScaler for feature scaling
        self.scaler = StandardScaler()
        self.train_data = None
        self.test_data = None
        self.experiment_log = []

    def prepare_data(self, test_size=0.2, random_state=42):
       
        # Separate features (X) and target (y)
        X = self.data.drop(columns=['target'])
        y = self.data['target']
        
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        
        # Feature scaling
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Store the train and test data
        self.train_data = (X_train_scaled, y_train)
        self.test_data = (X_test_scaled, y_test)
        
        # Log experiment details 
        experiment_details = {
            "test_size": test_size,
            "random_state": random_state,
            "scaler_mean": self.scaler.mean_.tolist(),
            "scaler_variance": self.scaler.var_.tolist()
        }
        self.experiment_log.append(experiment_details)

    def get_feature_stats(self):
       
        stats = self.data.describe().T[['mean', 'std', 'min', 'max']]
        return stats

# IrisExperiment Class
class IrisExperiment:
    def __init__(self, data_processor):
       
        self.data_processor = data_processor
        self.models = {
            "Logistic Regression": LogisticRegression(max_iter=200),
            "Random Forest": RandomForestClassifier(n_estimators=100)
        }
        self.metrics = {
            "accuracy": make_scorer(accuracy_score),
            "precision": make_scorer(precision_score, average="weighted"),
            "recall": make_scorer(recall_score, average="weighted")
        }

    def run_experiment(self):
       

        self.data_processor.prepare_data()
        X_train, y_train = self.data_processor.train_data

        # Iterate over each model to conduct experiments
        for model_name, model in self.models.items():
            with mlflow.start_run(run_name=model_name):
                # Cross-validation
                results = cross_validate(model, X_train, y_train, cv=5, scoring=self.metrics, return_train_score=False)

                # Average metrics across folds
                accuracy = np.mean(results["test_accuracy"])
                precision = np.mean(results["test_precision"])
                recall = np.mean(results["test_recall"])

                # Results
                self.log_results(model_name, accuracy, precision, recall)

                print(f"{model_name}: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}")

    def log_results(self, model_name, accuracy, precision, recall):
        
        # Log model parameters
        mlflow.log_param("model_name", model_name)

        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)

        mlflow.sklearn.log_model(self.models[model_name], model_name)

data_processor = IrisDataProcessor()
experiment = IrisExperiment(data_processor)
experiment.run_experiment()


2024/11/12 12:54:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression: Accuracy=0.9583, Precision=0.9669, Recall=0.9583


2024/11/12 12:54:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest: Accuracy=0.9500, Precision=0.9595, Recall=0.9500
